# <a id='toc1_'></a>[Convolutional Neural Network for Anime Image Classification](#toc0_)


**Table of contents**<a id='toc0_'></a>    
- [Convolutional Neural Network for Anime Image Classification](#toc1_)    
  - [Objectives](#toc1_1_)    
  - [Setup](#toc1_2_)    
    - [Installing Required Libraries](#toc1_2_1_)    
    - [Importing Required Libraries](#toc1_2_2_)    
  - [Load the Data](#toc1_3_)    
    - [Visualize the Dataset](#toc1_3_1_)    
  - [Define Custom Dataset Class](#toc1_4_)    
  - [Split Dataset into Training and Validation Sets](#toc1_5_)    
  - [Define the CNN Model](#toc1_6_)    
  - [Define Loss Function and Optimizer](#toc1_7_)    
  - [Train the Model](#toc1_8_)    
  - [Visualize the Training and Test Loss](#toc1_9_)    
  - [Exercises](#toc1_10_)    
    - [Exercise 1 - Change activation function to `leaky_relu`](#toc1_10_1_)    
    - [Exercise 2 - Increase the number of epochs and observe the training and validation loss](#toc1_10_2_)    
    - [Exercise 3 - Use different character classes for training and validation](#toc1_10_3_)    
  - [Authors](#toc1_11_)    
  - [Contributors](#toc1_12_)    


Estimated time needed: **30** minutes


This tutorial demonstrates how to build and train a **Convolutional Neural Network (CNN)** for image classification. The goal is to train a model that can classify images into the correct categories.


## <a id='toc1_1_'></a>[Objectives](#toc0_)

After completing this lab you will be able to:

 - Apply **Convolutional Neural Network** to classify images.


----


## <a id='toc1_2_'></a>[Setup](#toc0_)

For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/) for managing the data.
*   [`numpy`](https://numpy.org/) for mathematical operations.
*   [`matplotlib`](https://matplotlib.org/) for additional plotting tools.
*   [`sklearn`](https://scikit-learn.org/stable/) for machine learning and machine-learning-pipeline related functions.
*   [`torch`](https://pytorch.org/) for building and training the deep neural network.
*   [`torchvision`](https://pytorch.org/vision/stable/index.html) for computer vision tasks.
